In [1]:
#Proximal Policy Optimzization algorithm
#Split version
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow_probability as tfp
import random
from collections import deque
from ou_noise import OUNoise
import actor_PPO_net
import critic_PPO_net
import Environment as En
import os
from timeit import default_timer as timer   


reward_path = 'Monitoring/reward/'
if not os.path.exists(reward_path): os.makedirs(reward_path)

Noise_path = 'Monitoring/Noise/'
if not os.path.exists(Noise_path): os.makedirs(Noise_path)
    
Loss_path = 'Monitoring/Loss/'
if not os.path.exists(Loss_path): os.makedirs(Loss_path)


environment = En.env() 
_ = environment.reset()

alpha_critic = 0.9 

#Input & Output
state_nums = environment.state_num() 
action_nums = environment.action_setting() 

input_size_state= state_nums
input_size_action = state_nums
output_size_critic = 1 
output_size_actor =  action_nums


#Reinforcement learning parmeter
dis = 0.99 
buffer_memory = 50000 #Replay memory에 몇개를 넣을 것인가? (Buffer)
alpha_critic = 1.0
exploration_noise = OUNoise(input_size_action)
epsilon = 0.2 # for PPO CLIPPING


def train(main_actor, main_critic, train_batch, batch_size):

    #make empty stack
    state_stack = np.empty(0);  action_stack = np.empty(0)
    Q_target_stack = np.empty(0); Q_old_stack = np.empty(0)
    state_stack = np.reshape(state_stack, (0, input_size_state))
    action_stack = np.reshape(action_stack, (0, output_size_actor))
    Q_target_stack = np.reshape(Q_target_stack, (0, output_size_critic))
    Q_old_stack = np.reshape(Q_old_stack, (0, output_size_critic))    
    
    
    
    #----------------------------- Actor train -----------------------------#
    # Actor train part
    
    Loss_actor = 0.0
    
    for state, action, reward, next_state, done, old_log_prob in train_batch:
        
        #print(np.mean(action))
      
        #Old Q
        Q_old = main_critic.predict(state, action)         
        
        #make V_old
        V_old = Q_old - reward
      
        actor_loss, _  = main_actor.update(main_critic, state, next_state, action, old_log_prob, V_old, reward) #, main_actor   
        
        Loss_actor += actor_loss/len(train_batch)
    

    #----------------------------- Critic train -----------------------------#
    #Apprioxiomate TD error (Advantage)

    
    # Critic train part
    for state, action, reward, next_state, done, old_log_prob in train_batch:

        #Old Q
        Q_old = main_critic.predict(state, action)           
        
        #PPO - on-policy (main_actor)
        next_action, _ = main_actor.predict(next_state) 
        #next_action하고 log_prob반환하니까
        
        #target_Q
        Q_target = Q_old + alpha_critic*(reward + dis*(main_critic.predict(next_state, next_action)) - Q_old)     
 
        
        #Stacking process
        state_stack = np.vstack([state_stack, state])
        Q_target_stack = np.vstack([Q_target_stack, Q_target])
        Q_old_stack = np.vstack([Q_old_stack, Q_old])
        action_stack = np.vstack([action_stack, action])


    #Update critic model 
    Critic_loss, _  = main_critic.update(state_stack, action_stack, Q_target_stack)  

        
    return Critic_loss, Loss_actor




def main():

    st_step = 15 #action을 몇 time-step마다 취할 것인지에 대한 숫자
    state_step = 0
    record_frequency = 1
    step_deadline = 100
    update_freq = 1
    train_loop_epoch = 2 #PPO 
    max_episodes = 1000
    batch_size = 128
    buff_len = batch_size
    Loss_step = 0  

    
    # Replay buffer를 deque로 짠다. 
    buffer = deque() 

    reward_buffer = deque() #maxlen=100
    #reward_buffer또한 deque로 만들어서 마지막 100개까지 기억하도록 한다
    

    with tf.Session() as sess:
             
        #formation of network for actor net
        main_actor = actor_PPO_net.actor(sess, input_size_state, output_size_actor, output_size_critic, name="main_actor") 
       
        #formation of network for critic net (first error NameError - input_size ciritic 등)
        main_critic = critic_PPO_net.critic(sess, input_size_state, input_size_action, output_size_critic, main_actor, name="main_critic") 
        #main_critic에 main_actor를 넣어줘서 연결시킴 Q(s,a) - Objective
        
        _ = main_critic.initialization_c(name ="main_critic")
        _ = main_actor.initialization_a(main_critic.Objective, name ="main_actor")
        
        sess.run(tf.global_variables_initializer()) 
        print("initialization complete")    


        for episode in range(0, max_episodes+1):
            
            print("Episode : {} start ".format(episode))
        
            done = False
            
            ##################### environment로부터 state를 받아온다 (observation) ###############
            state = environment.reset() #envrionment로부터 state를 가져온다. (초기 state)
            exploration_noise.reset()       
            
            reward_graph = 0

            #Noise 그래프 그리기        
            
            reward_record = open(reward_path + "reward.plt" , 'a', encoding='utf-8', newline='') 
            noise_record = open(Noise_path + "noise, episode{}.plt" .format(episode), 'a', encoding='utf-8', newline='')
            state_reward_record = open(reward_path + "state_reward, episode{}.plt" .format(episode), 'a', encoding='utf-8', newline='')
            Loss_record = open(Loss_path + "Loss.plt".format(episode), 'a', encoding='utf-8', newline='')
            
            if episode ==0: 
                noise_record.write('VARIABLES = "state_step", "noise" \n') 
                state_reward_record.write('VARIABLES = "state_step", "avg_reward" \n')  
                reward_record.write('VARIABLES = "Episode", "Reward" \n') 
                Loss_record.write('VARIABLES = "state_step", "Loss" \n')
            
            
            while not done == True:

                Noise = 0.5*exploration_noise.noise() # 매 step마다 Normal distribution에서 임의로 추출한다.
                      
                #------------------------Stochastic policy part ------------------------#

                action, log_prob = main_actor.predict(state)
    
                #action_noise = action + (Noise) #with N(Noise) #이
                
                #noise를 입력한다
                noise_record.write("%d %f \n" %(state_step ,np.mean(Noise)))
                
                #Noise 조절이 좀 필요하다 step loop 안으로 들어와야 계속 변할수 있다. 
                #action_noise = np.reshape(action_noise, (input_size_critic_a))
                                
                # Get new state and reward from environment  
                next_state, reward, done, record = environment.simulation(state, action, st_step, step_deadline, episode)
                
                #한 Episode에서 순간 reward를 기록하기 위함
                state_reward_record.write("%d %f \n" %(state_step ,reward))
                

                #한 step의 reward씩 계속 reward_graph에 쌓는다. summation of reward
                reward_graph = reward + reward_graph

                #----------------------------- Replay buffer -------------------------#
                buffer.append((state, action, reward, next_state, done, log_prob))
                
                if len(buffer) > buffer_memory:
                    buffer.popleft()         
                    
                #--------------------------- Learning part ----------------------------#
                if len(buffer) > buff_len and state_step % update_freq == 0:

                    loss_avg = 0
                    
                    for _ in range(train_loop_epoch): #PPO에서는 train_loop_epoch을 활성화 시킨다.
                        
                        minibatch = random.sample(buffer, batch_size) 
                        minibatch = list(minibatch)
                        
                        #critic update start
                        loss_critic, loss_actor = train(main_actor, main_critic, minibatch, batch_size)
                        
                        loss_avg = loss_critic/train_loop_epoch +loss_avg

                    print("Loss for critic is : {}".format(loss_avg)) 
                    Loss_record.write("%d %f \n" %(Loss_step ,loss_avg))
                               
                #---------------------------------------------------------------------#       
               
                state = next_state
                
                state_step += 1
                Loss_step += 1
                #print("step num : {}".format(step))
                
                # break part
                if done==True or state_step == step_deadline:
                    break
               
            
            reward_graph = reward_graph/state_step
            
            state_step = 0 #state_step zero            
            
            #plt file로 reward graph 저장
            reward_record.write("%d %f \n" %(episode , reward_graph))
            
            noise_record.close()
            state_reward_record.close()
            Loss_record.close()
            reward_record.close()


if __name__ == "__main__":
    
    main()
    
    print("All process is finished!")




main_actor
Actor_net connected
main_critic
Critic_net connected - for action_feed
main_critic
Critic_net connected - for actor_feed
initialization complete
Episode : 0 start 
avg_reward
-0.037918306309335126
avg_reward
-0.08316348346822357
avg_reward
-0.14354868720882416
avg_reward
-0.23282762206568836
avg_reward
-0.2672465615670137
avg_reward
-0.3166624362662385
avg_reward
-0.34754056007871664
avg_reward
-0.39295225898542835
avg_reward
-0.4175366727923777
avg_reward
-0.4206974827878375
avg_reward
-0.37126249567371317
avg_reward
-0.2954937959458634
avg_reward
-0.3823870562199603
avg_reward
-0.3098845950090836
avg_reward
-0.3613051770053212
avg_reward
-0.18756872606172362
avg_reward
-0.275045638899122
avg_reward
-0.2939949797745364
avg_reward
-0.3630565136612709
avg_reward
-0.21645360896398433
avg_reward
-0.23508600929579954
avg_reward
-0.19678724939331105
avg_reward
-0.3366939072843059
avg_reward
-0.27720938689920144
avg_reward
-0.2432588218932416
avg_reward
-0.22769102820545675
avg_re

Loss for critic is : 0.03796885348856449
avg_reward
-0.07719353992810737
Loss for critic is : 0.0374018419533968
avg_reward
-0.0904433018729646
Loss for critic is : 0.03664250671863556
avg_reward
-0.07834511458101644
Loss for critic is : 0.03613210283219814
avg_reward
-0.08585352286994054
Loss for critic is : 0.03492768108844757
avg_reward
-0.0650633258435355
Loss for critic is : 0.034111570566892624
avg_reward
-0.10226551774484605
Loss for critic is : 0.0335321519523859
avg_reward
-0.10789550624426872
Loss for critic is : 0.03291033022105694
avg_reward
-0.113437292600756
Loss for critic is : 0.032458554953336716
avg_reward
-0.12542530195636747
Loss for critic is : 0.03183715231716633
avg_reward
-0.13566751832187846
Loss for critic is : 0.03114527277648449
avg_reward
-0.18348545759454002
Loss for critic is : 0.03080465830862522
avg_reward
-0.18996584578557218
Loss for critic is : 0.030354967340826988
avg_reward
-0.236703441917508
Loss for critic is : 0.030257634818553925
avg_reward
-0.

Loss for critic is : 0.04427293688058853
avg_reward
-1.0312402815390944
Loss for critic is : 0.039208345115184784
avg_reward
-0.8015922231060967
Loss for critic is : 0.04336860030889511
avg_reward
-0.8100689636367541
Loss for critic is : 0.03806553594768047
avg_reward
-0.8837284381981589
Loss for critic is : 0.043846312910318375
Episode : 3 start 
avg_reward
-0.044294640293493025
Loss for critic is : 0.03444484435021877
avg_reward
-0.07719256476518832
Loss for critic is : 0.032490719109773636
avg_reward
-0.10473224573402784
Loss for critic is : 0.04221835546195507
avg_reward
-0.12441547249987531
Loss for critic is : 0.05287611298263073
avg_reward
-0.14151141819641364
Loss for critic is : 0.037539007142186165
avg_reward
-0.1774321999330304
Loss for critic is : 0.024665926583111286
avg_reward
-0.25531796992828176
Loss for critic is : 0.023496929556131363
avg_reward
-0.24654209705898616
Loss for critic is : 0.02123444527387619
avg_reward
-0.3764352368979694
Loss for critic is : 0.02200669

Loss for critic is : 0.08973471820354462
avg_reward
-0.17397984706153685
Loss for critic is : 0.08447854965925217
avg_reward
-0.1811243005432529
Loss for critic is : 0.08686456084251404
avg_reward
-0.18516679814281622
Loss for critic is : 0.08391916751861572
avg_reward
-0.1840243278357267
Loss for critic is : 0.08255432173609734
avg_reward
-0.18957809117209223
Loss for critic is : 0.07789607346057892
avg_reward
-0.1736083544612157
Loss for critic is : 0.08031192421913147
avg_reward
-0.16692880629725812
Loss for critic is : 0.08068451657891273
avg_reward
-0.16200964496882594
Loss for critic is : 0.08026828244328499
avg_reward
-0.1445570368233002
Loss for critic is : 0.07555504888296127
avg_reward
-0.1260094171159455
Loss for critic is : 0.08516799658536911
avg_reward
-0.11021206647958462
Loss for critic is : 0.07858933880925179
avg_reward
-0.10053270423620907
Loss for critic is : 0.07310553640127182
avg_reward
-0.08407384872439017
Loss for critic is : 0.0770106315612793
avg_reward
-0.06

Loss for critic is : 0.07544330507516861
avg_reward
-0.18444382780429805
Loss for critic is : 0.0695846676826477
avg_reward
-0.19680646271000088
Loss for critic is : 0.06363070569932461
avg_reward
-0.20875929010830296
Loss for critic is : 0.06011044234037399
avg_reward
-0.2129417308229751
Loss for critic is : 0.05909576639533043
avg_reward
-0.22228497293069416
Loss for critic is : 0.05798843689262867
avg_reward
-0.25847506648128055
Loss for critic is : 0.0550918523222208
avg_reward
-0.24985601141970593
Loss for critic is : 0.054567232728004456
avg_reward
-0.26675867965733907
Loss for critic is : 0.05795029364526272
avg_reward
-0.2661273937227565
Loss for critic is : 0.05285531468689442
avg_reward
-0.2837471472021107
Loss for critic is : 0.05578869953751564
avg_reward
-0.2671592104980184
Loss for critic is : 0.054700881242752075
avg_reward
-0.32123952330733707
Loss for critic is : 0.050741540268063545
avg_reward
-0.29278405583041944
Loss for critic is : 0.05021017789840698
avg_reward
-0

...

Loss for critic is : 0.007015163544565439
avg_reward
-0.014592979449461706
Loss for critic is : 0.0067328400909900665
avg_reward
-0.015103181682265694
Loss for critic is : 0.006734557216987014
avg_reward
-0.013474785275464269
Loss for critic is : 0.006808114005252719
avg_reward
-0.012981897662039067
Loss for critic is : 0.00600661407224834
avg_reward
-0.015796907691548377
Loss for critic is : 0.006358656566590071
Episode : 97 start 
avg_reward
-0.009382641791909622
Loss for critic is : 0.006353959906846285
avg_reward
-0.016492674564702884
Loss for critic is : 0.0062662409618496895
avg_reward
-0.02255421710389071
Loss for critic is : 0.006559888366609812
avg_reward
-0.015708931580241996
Loss for critic is : 0.0059806841891258955
avg_reward
-0.025779322658775108
Loss for critic is : 0.006019810913130641
avg_reward
-0.016916737792764862
Loss for critic is : 0.006704902276396751
avg_reward
-0.017633746674137765
Loss for critic is : 0.007206027861684561
avg_reward
-0.01524812979625659
Loss 

Loss for critic is : 0.006437073461711407
avg_reward
-0.018964083163306382
Loss for critic is : 0.006429280852898955
avg_reward
-0.02116705814049284
Loss for critic is : 0.00626359460875392
avg_reward
-0.01570193599969212
Loss for critic is : 0.006736418465152383
avg_reward
-0.017105144377115536
Loss for critic is : 0.006534544052556157
avg_reward
-0.01736511361559607
Loss for critic is : 0.006108832778409123
avg_reward
-0.015298779086527617
Loss for critic is : 0.006483662640675902
avg_reward
-0.02069177144824462
Loss for critic is : 0.006580141838639975
avg_reward
-0.02055921353709652
Loss for critic is : 0.0064267488196492195
avg_reward
-0.021699627261015597
Loss for critic is : 0.0073451693169772625
avg_reward
-0.019394096722625116
Loss for critic is : 0.007022356381639838
avg_reward
-0.02604631089787393
Loss for critic is : 0.006489879917353392
avg_reward
-0.029584606472489103
Loss for critic is : 0.006393552292138338
avg_reward
-0.026976045691340836
Loss for critic is : 0.0065491

Loss for critic is : 0.006294812774285674
avg_reward
-0.016889509179747336
Loss for critic is : 0.006884399801492691
avg_reward
-0.011650853906767704
Loss for critic is : 0.006390666589140892
avg_reward
-0.01487493242450872
Loss for critic is : 0.006573175545781851
avg_reward
-0.01785736961170696
Loss for critic is : 0.00652065291069448
avg_reward
-0.019282219902876423
Loss for critic is : 0.006317767547443509
avg_reward
-0.018115705664681153
Loss for critic is : 0.006289559183642268
avg_reward
-0.02293884547161927
Loss for critic is : 0.006489145802333951
avg_reward
-0.024840865119737956
Loss for critic is : 0.006111315917223692
avg_reward
-0.022140484460841246
Loss for critic is : 0.006167807849124074
avg_reward
-0.020067904736371617
Loss for critic is : 0.007014100672677159
avg_reward
-0.014400354438833151
Loss for critic is : 0.006812560837715864
avg_reward
-0.012061659523808891
Loss for critic is : 0.00664070644415915
avg_reward
-0.015131680053767112
Loss for critic is : 0.0063543

Loss for critic is : 0.0062292448710650206
avg_reward
-0.009385115854873026
Loss for critic is : 0.006897986866533756
avg_reward
-0.01349972162864866
Loss for critic is : 0.00673944759182632
avg_reward
-0.009438909192607542
Loss for critic is : 0.006653988733887672
avg_reward
-0.005390814397212947
Loss for critic is : 0.0065273328218609095
avg_reward
-0.004421416552573738
Loss for critic is : 0.006690433248877525
avg_reward
-0.0026174394325264166
Loss for critic is : 0.0066118426620960236
avg_reward
0.0014184606899632062
Loss for critic is : 0.006788827478885651
avg_reward
-0.004522403106328872
Loss for critic is : 0.006915516685694456
avg_reward
-0.0010783049805140645
Loss for critic is : 0.006548123434185982
avg_reward
-0.003157646461280448
Loss for critic is : 0.007448915392160416
avg_reward
-0.009141338999755742
Loss for critic is : 0.007046971935778856
avg_reward
-0.007582708380529933
Loss for critic is : 0.006662801373749971
avg_reward
-0.0030264553986153395
Loss for critic is : 

KeyboardInterrupt: 